# Finding Antares SNe in DESI BGS # 

#### This notebook selects BGS in coadd tiles and find SNe from Antares in those BGS positions ####

In [42]:
import numpy as np

from astropy.io import fits
from astropy.table import Table, vstack
import matplotlib.pyplot as plt

import desitarget
from desitarget.targetmask import desi_mask, bgs_mask
from desitarget.cmx.cmx_targetmask import cmx_mask
from desispec.io import read_spectra

from glob import glob

from astropy.coordinates import SkyCoord, match_coordinates_sky
from astropy.io import ascii
from astropy import units as u


### example single coadd file ###

In [2]:
coadd_file = '/global/cfs/cdirs/desi/spectro/redux/daily/tiles/70502/20200225/coadd-0-70502-20200225.fits' # spectra in petal 0 in 70502 tile observed on Feb. 25 2020 

In [4]:
coaddsfits=fits.open(coadd_file)

In [5]:
coaddsfits.info()

Filename: /global/cfs/cdirs/desi/spectro/redux/daily/tiles/70502/20200225/coadd-0-70502-20200225.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      34   ()      
  1  FIBERMAP      1 BinTableHDU    283   500R x 136C   [K, I, J, K, J, J, D, D, E, E, E, E, E, E, K, B, 3A, E, E, I, J, D, J, J, I, J, 8A, J, 4A, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, I, I, I, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, I, I, I, E, E, E, E, E, E, B, B, I, E, E, E, E, E, E, E, E, E, E, E, E, E, E, K, 2A, E, E, E, E, E, E, E, E, L, E, L, E, E, 1A, K, K, K, K, E, E, E, E, K, K, K, K, D, D, D, D, D, D, D, D, K, K, K]   
  2  BRZ_WAVELENGTH    1 ImageHDU        10   (7781,)   float64   
  3  BRZ_FLUX      1 ImageHDU        11   (7781, 500)   float32   
  4  BRZ_IVAR      1 ImageHDU        11   (7781, 500)   float32   
  5  BRZ_MASK      1 ImageHDU        12   (7781, 500)   int32 (rescales to uint32)   
  6  BRZ_RESOLUTION    1 

In [6]:
coaddsfits['fibermap'].header

XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                  611 / length of dimension 1                          
NAXIS2  =                  500 / length of dimension 2                          
PCOUNT  =                    0 / number of group parameters                     
GCOUNT  =                    1 / number of groups                               
TFIELDS =                  136 / number of table fields                         
TTYPE1  = 'TARGETID'           / Unique target ID                               
TFORM1  = 'K       '                                                            
TTYPE2  = 'PETAL_LOC'          / Petal location [0-9]                           
TFORM2  = 'I       '                                                            
TTYPE3  = 'DEVICE_LOC'      

### loop through all coadds in one tile ###

In [7]:
prefix = '/global/cfs/cdirs/desi/spectro/redux/daily/tiles/70502/20200225'
coadds = sorted(glob('{}/*coadd*.fits'.format(prefix)))
coadds

['/global/cfs/cdirs/desi/spectro/redux/daily/tiles/70502/20200225/coadd-0-70502-20200225.fits',
 '/global/cfs/cdirs/desi/spectro/redux/daily/tiles/70502/20200225/coadd-3-70502-20200225.fits',
 '/global/cfs/cdirs/desi/spectro/redux/daily/tiles/70502/20200225/coadd-6-70502-20200225.fits',
 '/global/cfs/cdirs/desi/spectro/redux/daily/tiles/70502/20200225/coadd-7-70502-20200225.fits',
 '/global/cfs/cdirs/desi/spectro/redux/daily/tiles/70502/20200225/coadd-9-70502-20200225.fits']

In [9]:
fibermap = None
for coadd in coadds:
    hdus = fits.open(coadd)
    spectra = read_spectra(coadd)
    if fibermap is None:
        fibermap = spectra.fibermap  # read in the first coadd fibermap
    else:
        fibermap = vstack([fibermap, spectra.fibermap])

### select BGS ###

In [76]:
cmx_mask.mask('SV0_BGS') # this is commissioning SV0 BGS mask


256

In [20]:
sv0_bgs= fibermap['CMX_TARGET'] & cmx_mask.mask('SV0_BGS') != 0


In [77]:
np.sum(sv0_bgs) # total 1484 BGS in 5 petals

1484

In [25]:
columns = ['TARGETID', 'TARGET_RA', 'TARGET_DEC']

In [34]:
fibermap[sv0_bgs][columns]

TARGETID,TARGET_RA,TARGET_DEC
int64,float64,float64
35185712445788791,179.7394266438731,-1.9417105015239313
35185712449979800,179.8594279099647,-2.0776414098954636
35185712449979886,179.89090909750206,-2.073698199126241
35185712449980146,179.82183059798993,-2.063184299691656
35185712449980558,179.91006848009278,-2.0472234852725726
35185712449980961,179.88092433607022,-2.033094124924236
35185712449980984,179.83198680339055,-2.03213302095459
35185712449981157,179.93672929064755,-2.0249141792900964
35185712449981248,179.98371902847697,-2.0218686397027863


In [62]:
BGS_targets=fibermap[sv0_bgs]

In [61]:
print(np.max(fibermap[sv0_bgs]['TARGET_DEC']))
print(np.median(fibermap[sv0_bgs]['TARGET_DEC']))

1.0093919996327139
-0.3286756513269453


In [29]:
coadds_ra  = fibermap[sv0_bgs]['TARGET_RA']
coadds_dec = fibermap[sv0_bgs]['TARGET_DEC']


### Matching BGS coords with with Antares SNe coords ###

In [32]:
coadds_coords = SkyCoord(ra=coadds_ra, dec=coadds_dec, unit='deg')


In [67]:
antares_data = ascii.read('/global/u1/x/xychen/timedomain/antares/results.csv') # to generate this file, run python query_alerts.py --ra XX --dec XX --tobs XXXXX (ra, dec in degree that are close to tile ra and dec, tobs in MJD) in a directory where the antares client python package is installed

In [68]:
columns_antares=['ra','dec']

In [69]:
print(antares_data[columns_antares])

     ra        dec    
----------- ----------
180.3798287 11.1097697
180.0149061 10.3722928


In [70]:
antares_coords = SkyCoord(ra=antares_data['ra'], dec=antares_data['dec'], unit='deg')


In [71]:
idx, sep2d, dist3d = match_coordinates_sky(coadds_coords, antares_coords)


In [72]:
selected_targets = None

select = sep2d < 3600*10*u.arcsec # this is 10 degree, made it large to find more matches for demonstration. 1 arcsec is more reasonable

selected_targets = BGS_targets[select] 



In [73]:
print(np.sum(select), len(selected_targets)) # 150 matched in this case

150 150


In [74]:
print(selected_targets)

     TARGETID     PETAL_LOC DEVICE_LOC ... FIRST_FIBER LAST_FIBER NUM_FIBER
----------------- --------- ---------- ... ----------- ---------- ---------
35185772856344986         6        166 ...        3395       3395         1
35185772856344991         6        136 ...        3393       3393         1
35185772856345632         6        180 ...        3357       3357         1
35185772856345785         6        149 ...        3366       3366         1
35185772856347508         6        194 ...        3018       3018         1
35185772856348235         6        193 ...        3023       3023         1
35185772856348738         6        226 ...        3009       3009         1
35185772856349566         6        243 ...        3036       3036         1
35185772856349791         6        224 ...        3035       3035         1
35185772856350155         6        260 ...        3495       3495         1
              ...       ...        ... ...         ...        ...       ...
351857849401

In [75]:
print(select)

[False False False ... False False False]
